In [2]:
import pandas as pd
data=pd.read_csv("netfix_cleaned.csv")
data.head()

,names,release_year,maturity_rating,duration,description,genre,mood,cast,subtitles,audio
0,Mission Majnu,2023,U/A 16+,2h 9m,"In the 1970s, an undercover Indian spy takes o...","['Spy Movies', 'Hindi-Language Movies', 'Bolly...",Suspenseful,"['Sidharth Malhotra', 'Rashmika Mandanna', 'Pa...","['English,', 'English,', 'Hindi']","['English,', 'Hindi - Audio Description,', 'Hi..."
1,Cirkus,2022,U/A 7+,2h 14m,Chaos and comedy take the spotlight when a rin...,"['Hindi-Language Movies', 'Bollywood Movies', ...",Goofy,"['Ranveer Singh', 'Varun Sharma', 'Pooja Hegde...","['English,', 'Hindi']",['Hindi [Original]']
2,Gangubai Kathiawadi,2022,U/A 16+,2h 33m,"Duped and sold to a brothel, a young woman fea...","['Hindi-Language Movies', 'Movies Based on Boo...",Provocative,"['Alia Bhatt', 'Vijay Raaz', 'Seema Pahwa', 'A...","['English,', 'Hindi']","['Hindi [Original],', 'Telugu']"
3,Thunivu,2023,U/A 16+,2h 22m,A major bank heist takes an unnerving turn whe...,"['Crime Movies', 'Action & Adventure']",Exciting,"['Ajith Kumar', 'Manju Warrier', 'Samuthirakan...","['English,', 'Hindi']",['Tamil [Original]']
4,Bhool Bhulaiyaa 2,2022,U/A 13+,2h 21m,"When strangers Reet and Ruhan cross paths, the...","['Hindi-Language Movies', 'Bollywood Movies', ...","Offbeat,","['Tabu', 'Kartik Aaryan', 'Kiara Advani', 'Raj...","['English,', 'Hindi']",['Hindi [Original]']


In [3]:
import numpy as np
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
print(data.isnull().sum())

names               0
release_year        0
maturity_rating     0
duration            0
description         0
genre               0
mood               43
cast                0
subtitles           0
audio               0
dtype: int64


In [5]:
data = data[["names", "description", "maturity_rating", "genre"]]
print(data.head())

                 names                                        description  \
0        Mission Majnu  In the 1970s, an undercover Indian spy takes o...   
1               Cirkus  Chaos and comedy take the spotlight when a rin...   
2  Gangubai Kathiawadi  Duped and sold to a brothel, a young woman fea...   
3              Thunivu  A major bank heist takes an unnerving turn whe...   
4    Bhool Bhulaiyaa 2  When strangers Reet and Ruhan cross paths, the...   

  maturity_rating                                              genre  
0        U/A 16+   ['Spy Movies', 'Hindi-Language Movies', 'Bolly...  
1         U/A 7+   ['Hindi-Language Movies', 'Bollywood Movies', ...  
2        U/A 16+   ['Hindi-Language Movies', 'Movies Based on Boo...  
3        U/A 16+              ['Crime Movies', 'Action & Adventure']  
4        U/A 13+   ['Hindi-Language Movies', 'Bollywood Movies', ...  


In [6]:
data=data.dropna()

In [7]:
import nltk
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["names"] = data["names"].apply(clean)

[nltk_data] Downloading package stopwords to C:\Users\aayushi
[nltk_data]     sinha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
print(data.names.sample(10))

275                  spiderman way home
543                       kantara hindi
179                               stray
257                           raw hindi
322    chhota bheem  krishna mayanagari
206                        stuart littl
329                           tom jerri
417                                moon
266                        sarkar tamil
319                               invit
Name: names, dtype: object


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

features = data["genre"].tolist()

vectorizer = TfidfVectorizer(stop_words="english")
matrix = vectorizer.fit_transform(features)
similarity = cosine_similarity(matrix)


In [13]:
data.tail(3)

,names,description,maturity_rating,genre
557,major telugu,This biopic follows the life of Indian Army of...,U/A 16+,"['Movies Based on Real Life', 'Action & Advent..."
558,om shanti om,Reincarnated 30 years after being killed in a ...,U/A 16+,"['Romantic Comedies', 'Hindi-Language Movies',..."
559,kabhi khushi kabhi gham,Years after his father disowns his adopted bro...,U/A 13+,"['Hindi-Language Movies', 'Bollywood Movies', ..."


In [49]:


indices = pd.Series(data.index,index=data['names']).drop_duplicates()

In [14]:

def netflix_recommendation(Names,similarity_matrix, indices, data):
    if names in indices:
        index = indices[names]
        similarity_scores = similarity_matrix[index]
        top_indices = similarity_scores.argsort()[::-1][:10]
        recommended_movies = data.loc[top_indices, 'names']
        return recommended_movies
        print(f"movie '{names}' found")
    else:
        print(f"Movie '{names}' not found in the dataset.")
        return None

movie_name = "Cirkus"
recommendations = netflix_recommendation(names, similarity, indices, data)
if recommendations is not None:
    print(recommendations)


NameError: name 'names' is not defined